# DDL e DML

O SQL usa os termos **tabela**, **linha** e **coluna** para os termos do modelo relacional formal *relação*, *tupla* e *atributo* respectivamente.

In [13]:
# Utilizaremos a lib csvms em nossos exemplos
from csvms.table import Table
#For re-run
import shutil
from csvms.schema import Database
shutil.rmtree(Database.FILE_DIR, ignore_errors=True)

## Criação

O comando `CREATE TABLE` é usado para especificar uma nova relação, dando-lhe um **nome** e especificando seus **atributos**. Cada um dos atributos recebe um **nome** e um **tipo de dado** para especificar seu domínio de valores.

```sql
CREATE TABLE lista_frutas (
    nm_fruta TEXT ,
    tp_fruta TEXT )
```

> Como nosso banco de dados será escrito na linguagem [python](https://www.w3schools.com/python/default.asp), utilizaremos os seus [tipos primitivos](https://www.w3schools.com/python/python_datatypes.asp) como tipos de dados.

Os tipos de dados básicos disponíveis para atributos costumam ser:

+ Numéricos
+ Cadeia de caracters
+ Cadeia de bits
+ Booleano
+ Data
+ Hora

> E é já a partir deste ponto em que os vários dialetos começam a surgir...

![create table](../imgs/create_table.png)

In [14]:
Table(
    name="lista_frutas",
    columns={
        'nm_fruta':str,
        'tp_fruta':str
    }
).save()  

INFO:csvms:creating new data dictionary in data/catalog.json


True

## Transação

O inicio do trabalho `BEGIN` se dá com o carregamento da tabela do sistema de arquivos para a SGA (System Global Area).
Tudo o que ocorrer com este objeto durante seu período em mória será considerado uma transação até que se confirme com os comandos de `COMMIT` ou `ROLLBACK`

In [15]:
lf=Table("lista_frutas")

## Inclusão

Em sua forma mais simples, o comando `INSERT` é usado para acrescentar uma única *tupla* (linha) em uma *relação* (tabela).
Precisamos especificar o nome da relação e uma lista de valores para a tupla. Os valores devem ser informados na mesma ordem em que os atributos correspondentes foram especificados no `CREATE TABLE`.

![insert](../imgs/insert.png)

```sql
INSERT INTO lista_frutas VALUES ('banana','doce');
INSERT INTO lista_frutas VALUES ('limão','azedo');
INSERT INTO lista_frutas VALUES ('maçã','doce');
```

In [16]:
lf.append('banana','doce')
lf.append('limão','azedo')
lf.append('maçã','doce')
print(lf)

INFO:csvms:Row inserted
INFO:csvms:Row inserted
INFO:csvms:Row inserted


   TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|    limão|    azedo|
  2|     maçã|     doce|
   +---------+---------+



## Confirmação

Com o comando de `COMMIT` confirmamos que todas as alterações desta transação deverão ser **salvas** em definitivo.

```sql
COMMIT
```

In [17]:
lf.save()

True

## Exclusão

O comando `DELETE` remove *tuplas* de uma relação. Ele possui uma clausula *`WHERE`*, semelhante a utilzada em consultas, para selecionar a tuplas que serão excluídas.

```sql
DELETE FROM lista_frutas WHERE nm_fruta = 'limão'
```

In [6]:
# Procura em toda a tabela por linhas cujo nome da fruta seja "limão"
for idx in range(len(lf)):
    if lf[idx]["nm_fruta"]=="limão":
        del lf[idx]
print(lf)

INFO:csvms:Row deleted


   TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|     maçã|     doce|
   +---------+---------+



## Atualização

O comando `UPDATE` é utilizado para modificar valores de atributos de uma ou mais *tuplas* selecionadas. Assim como no comando `DELETE`, é possível adicionar uma clausula `WHERE` no comando para informar o critério de seleção das linhas que serão afetadas.

```sql
UPDATE lista_frutas 
   SET tp_fruta = 'doce'
 WHERE tp_fruta = 'amargo'
```

In [7]:
for idx in range(len(lf)):
    if lf[idx]["tp_fruta"]=='doce':
        row = lf[idx]
        row["tp_fruta"]='amargo'
        lf[idx] = tuple(row.values())
print(lf)

INFO:csvms:Row updated
INFO:csvms:Row updated


   TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|   amargo|
  1|     maçã|   amargo|
   +---------+---------+



## Descarte

Com o comando de `ROLLBACK` descartamos todas as alterações desta transação, voltando assim ao estado em que os dados se encontravam desde a ultima operação salva. O comando de `ROLLBACK` finalizará um trabalho e iniciará outro.

```sql
ROLLBACK
```

In [8]:
# Recarrega todas as tabelas da SGA
lf = Table('lista_frutas')

## Alteração

O comando `ALTER TABLE` serve para alterar os atributos definidos para a tabela durante o processo de criação `CREATE TABLE`. Seus principais operadores servem para adição (`ADD`), modificação (`MODIFY`) tipo de dado, renomear (`RENAME`) um atributo ou ainda a remoção (`DROP`) de atributos existentes.

```sql
-- Adiciona uma nova coluna do tipo inteiro chamada valor
ALTER TABLE lista_frutas ADD (valor INTEGER)
```
```sql
-- Renomear a coluna para a convenção de nomes utilizada na tabela
-- e alterar seu tipo para ponto flutuante 
ALTER TABLE lista_frutas MODIFY (valor INTEGER) TO (vl_fruta FLOAT);
```
```sql
-- Remove a coluna 'vl_fruta' da tabela
ALTER TABLE lista_frutas DROP vl_fruta;
```

> Este sistema não prevê *concorrencia*, por isso podemos trabalhar sobre os objetos da SGA

In [9]:
lf.alter('ADD',{'valor':int}).save()
print(lf)

   TABLE: default.lista_frutas
   +---------+---------+------+
   |nm_fruta |tp_fruta |valor |
   +---------+---------+------+
  0|   banana|     doce|     0|
  1|    limão|    azedo|     0|
  2|     maçã|     doce|     0|
   +---------+---------+------+



In [10]:
lf.alter('MODIFY',{'valor':int},{'vl_fruta':float}).save()
print(lf)

   TABLE: default.lista_frutas
   +---------+---------+---------+
   |nm_fruta |tp_fruta |vl_fruta |
   +---------+---------+---------+
  0|   banana|     doce|      0.0|
  1|    limão|    azedo|      0.0|
  2|     maçã|     doce|      0.0|
   +---------+---------+---------+



In [11]:
lf.alter('DROP',{'vl_fruta':float}).save()
print(lf)

   TABLE: default.lista_frutas
   +---------+---------+
   |nm_fruta |tp_fruta |
   +---------+---------+
  0|   banana|     doce|
  1|    limão|    azedo|
  2|     maçã|     doce|
   +---------+---------+



## Eliminação

Como o nome diz, o comando `DROP TABLE` eliminará o objeto do sistema. Ou seja, tanto o que está em memória quanto no sistema de arquivos e catalogo de dados.

![drop table](../imgs/drop_table.png)

In [12]:
lf.drop()

True